In [2]:
import pandas as pd 
import numpy as np
import requests 
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
!pip install folium
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 94 kB 5.7 MB/s  eta 0:00:01


In [3]:
req = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1012118802') 
soup = BeautifulSoup(req.content,'lxml') 
table = soup.find_all('table')[0]  
df = pd.read_html(str(table)) 
df_neighbourhood = pd.DataFrame(df[0])

In [4]:
# Delete not assigned Boroughs
df_neighbourhood = df_neighbourhood[df_neighbourhood['Borough'] != 'Not assigned']

In [5]:
# Amount of neighbourhoods with Not assigned
df_neighbourhood[df_neighbourhood.Neighbourhood == 'Not assigned'].count()

Postal Code      0
Borough          0
Neighbourhood    0
dtype: int64

In [6]:
# Check if there are duplicities in Postal code (True -> unique values)
df_neighbourhood['Postal Code'].unique().shape[0] == df_neighbourhood.shape[0]

True

In [7]:
df_neighbourhood.head(5)

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df_neighbourhood.shape

(103, 3)

Question 2

In [9]:
# Geoloader does not work
'''
import geocoder
saved_column = df_neighbourhood['Postal Code']
saved_column.head()
lat_lng_coords = None
while(lat_lng_coords is None):

   g = geocoder.google('{}, Toronto, Ontario'.format(saved_column))

   lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]

longitude = lat_lng_coords[1]

print(latitude, longitude)
'''

"\nimport geocoder\nsaved_column = df_neighbourhood['Postal Code']\nsaved_column.head()\nlat_lng_coords = None\nwhile(lat_lng_coords is None):\n\n   g = geocoder.google('{}, Toronto, Ontario'.format(saved_column))\n\n   lat_lng_coords = g.latlng\n\nlatitude = lat_lng_coords[0]\n\nlongitude = lat_lng_coords[1]\n\nprint(latitude, longitude)\n"

In [10]:
url_csv = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv'
geo =pd.read_csv(url_csv)

In [11]:
df_geo = df_neighbourhood.merge(geo, on='Postal Code', how='left')
df_geo.head(5)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Question 3

In [12]:
mapTor = folium.Map(location=[df_geo.Latitude[0], df_geo.Longitude[0]], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df_geo['Latitude'], df_geo['Longitude'], df_geo['Borough'], df_geo['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapTor)  
    
mapTor

Foursquare venues

In [15]:
CLIENT_ID = '...' # Foursquare ID
CLIENT_SECRET = '...' # Foursquare Secret
VERSION = '20180605' # API version

In [16]:
df2.set_index('PostalCode', inplace = True) 
neighborhood_latitude = df2.loc['M9V']['Latitude']
neighborhood_longitude = df2.loc['M9V']['Longitude']

NameError: name 'df2' is not defined

In [17]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

NameError: name 'neighborhood_latitude' is not defined

In [19]:
results = requests.get(url).json()

NameError: name 'url' is not defined

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']

NameError: name 'results' is not defined

In [21]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

NameError: name 'json_normalize' is not defined

In [22]:

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

NameError: name 'nearby_venues' is not defined